In [1]:
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper
import topologicpy
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.Dictionary import Dictionary
from topologicpy.Cluster import Cluster
from src.topologicpy.Edge import Edge
from src.topologicpy.CellComplex import CellComplex

print(Helper.Version())
print(topologicpy.__file__)
renderer = "browser"

The version that you are using (0.8.15) is the latest version available on PyPI.
/Users/roger/miniconda3/lib/python3.12/site-packages/topologicpy/__init__.py


# Cube Grid Function

In [2]:
building_complex = CellComplex.Box(width=3, length=3, height=3, uSides=3, vSides=3, wSides=3)
print(building_complex)

Topology.Show(building_complex, renderer=renderer)

In [21]:
from topologicpy.Topology import Topology
from topologicpy.CellComplex import CellComplex
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face


def create_environment_shell(building_complex):
    """
    Creates an environment shell centered around `building_complex` with a **horizontal separation**.
    Returns both the repositioned `building_complex` and `env_shell`.

    Parameters:
        building_complex (topologic_core.CellComplex): The building.

    Returns:
        tuple: (repositioned_building_complex, env_shell)
    """
    width, length, height = 4.0, 4.0, 4.0  # Environment box size

    lower_ratio = 0.3  # 20% lower part
    upper_ratio = 1.0 - lower_ratio  # 80% upper part

    lower_height = height * lower_ratio
    upper_height = height * upper_ratio

    # Compute bounding box of the building
    building_bb = Topology.BoundingBox(building_complex)
    bb_vertices = Topology.Vertices(building_bb)

    min_z = min(Vertex.Z(v) for v in bb_vertices)
    max_z = max(Vertex.Z(v) for v in bb_vertices)
    building_center = Topology.Centroid(building_complex)

    # Compute the vertical center shift
    building_height = max_z - min_z
    building_center_z = (max_z + min_z) / 2  # Midpoint of the building height
    env_bottom = building_center_z - height / 2  # Centering env_complex

    # Move `building_complex` to align its center with `env_complex`
    building_offset_z = -building_center_z  # Move building to world origin
    repositioned_building_complex = Topology.Translate(building_complex, 0, 0, building_offset_z)

    # Create full environment shell
    env_full = CellComplex.Box(
        width=width, length=length, height=height,
        uSides=1, vSides=1, wSides=1  # Single cell before splitting
    )

    # Compute the exact shift for correct alignment
    env_offset_z = env_bottom - min(Vertex.Z(v) for v in Topology.Vertices(env_full))

    # Move env_complex to match the repositioned `building_complex`
    env_complex = Topology.Translate(env_full, 0, 0, env_offset_z)

    # Create the horizontal split at **20% height**
    split_height = env_bottom + lower_height  # The split should be 20% up
    split_face = Face.Rectangle(
        origin=Vertex.ByCoordinates(0, 0, split_height),
        width=width * 1.1, length=length * 1.1
    )

    # Slice the environment shell at this height
    sliced_env = Topology.Slice(env_complex, split_face)

    # Ensure both halves exist
    env_cells = CellComplex.Cells(sliced_env)
    if not env_cells:
        print("⚠️ Error: No cells found after slicing. Returning original environment shell.")
        return repositioned_building_complex, env_complex

    # Create a new shell from the separated parts
    env_shell = Topology.Difference(sliced_env, repositioned_building_complex)

    return repositioned_building_complex, env_shell

In [40]:
building_complex, env_complex = create_environment_shell(building_complex)

building_graph = Graph.ByTopology(building_complex, viaSharedTopologies=True, toExteriorTopologies=True)
env_graph = Graph.ByTopology(env_complex, toExteriorTopologies=True)

print(env_complex)
print(building_complex)
print(building_graph)
print(env_graph)

Topology.Show(env_complex, env_graph, renderer=renderer)
Topology.Show(building_complex, building_graph, renderer=renderer)

# TODO: Brauchen eigene Graph Funktion basierend auf Centroids oder in Wand (Boden / Aussen) als Zonen speichern?

In [42]:
Topology.Show(env_complex, building_graph, env_graph, renderer=renderer)

In [39]:
building_graph = Graph.ByTopology(
    building_complex,
    viaSharedTopologies=True,
    toExteriorTopologies=True
)
env_graph = Graph.ByTopology(
    env_complex,
    toExteriorTopologies=True
)

b_topo = Graph.Topology(building_graph)
e_topo = Graph.Topology(env_graph)

merged_topo = Topology.Merge(b_topo, e_topo, tolerance=0.001)
merged_graph = Graph.ByTopology(merged_topo)

Topology.Show(env_complex, merged_graph, renderer=renderer)

In [51]:
def _flatten_topologies(input_topos, flattenType="Cell"):
    """
    Zerlegt die eingehenden Topologien in 'flattenType'-Untereinheiten. 
    Kann z. B. "Cell" oder "Face" sein. Je nachdem erhältst du eine Liste 
    aller Cells oder Faces aus dem jeweiligen Topology-Objekt.
    """
    result = []
    for topo in input_topos:
        if not Topology.IsInstance(topo, "Topology"):
            continue

        if flattenType.lower() == "cell":
            # Alle Zellen (Cells) aus dem Topology extrahieren
            sub_cells = Topology.Cells(topo)
            result.extend(sub_cells)
        elif flattenType.lower() == "face":
            # Alle Faces extrahieren
            sub_faces = Topology.Faces(topo)
            result.extend(sub_faces)
        else:
            # Fallback: falls ein anderer String oder None, 
            # einfach das Original-Objekt einfügen
            result.append(topo)

    return result


def by_topologies(topologies, flatten=True, flattenType="Cell", tolerance=0.0001):
    """
    Erzeugt einen Graphen auf Basis der übergebenen Topologien.
    Jeder Graph-Knoten wird jeweils der Zentroid (CenterOfMass) einer 
    aufgesplitteten (Sub-)Topologie (z.B. einer einzelnen Cell oder Face).

    standard: flatten=True, flattenType="Cell"
      => pro Input-Topologie => alle Cells => 
         jeder Node = Zentroid jeder Cell,
         Edges = wenn sich 2 Cells eine Face teilen.

    Parameter
    ---------
    topologies : list
        Liste aus Topology-Objekten (z.B. ein CellComplex oder ein Cluster).
    flatten : bool
        Falls True, zerlegen wir die eingehenden Topologien in 
        'flattenType'-Untereinheiten (Cell/Face).
    flattenType : str
        "Cell" oder "Face" (oder "Wire", etc.). Wie du willst.
    tolerance : float
        Toleranzwert für Edge-Konstuktion.

    Returns
    -------
    topologic.Graph
        Ein Graph mit je einem Knoten pro (Sub-)Topologie,
        Kanten, falls sich 2 Sub-Topologien eine Face teilen.
    """

    # (A) Vorverarbeitung / Zerlegung
    if flatten:
        topologies = _flatten_topologies(topologies, flattenType=flattenType)

    # (B) Liste (Topology, Zentroid)
    centroid_vertices = []
    for topo in topologies:
        if not Topology.IsInstance(topo, "Topology"):
            continue
        centroid = Topology.CenterOfMass(topo)
        centroid_vertices.append((topo, centroid))

    # (C) Kanten anlegen (Adjacency: Shared Faces)
    n = len(centroid_vertices)
    edges = []
    visited_pairs = set()

    for i in range(n):
        topo_i, cent_i = centroid_vertices[i]
        faces_i = Topology.Faces(topo_i)

        for j in range(i + 1, n):
            topo_j, cent_j = centroid_vertices[j]
            if (i, j) in visited_pairs or (j, i) in visited_pairs:
                continue

            # Prüfe, ob topo_i und topo_j mindestens eine Face gemeinsam haben
            shared_something = False
            for fi in faces_i:
                shared_faces = Topology.SharedFaces(fi, topo_j)
                if len(shared_faces) > 0:
                    # => JA, sie teilen sich was
                    shared_something = True
                    break

            if shared_something:
                e = Edge.ByVertices([cent_i, cent_j], tolerance=tolerance)
                edges.append(e)

            visited_pairs.add((i, j))
            visited_pairs.add((j, i))

    # (D) Bau den Graph
    only_vertices = [cv[1] for cv in centroid_vertices]
    g = Graph.ByVerticesEdges(only_vertices, edges)
    return g

In [56]:

# 1) Beispiel: Du hast eine Liste mit CellComplex-Objekten
big_comps = [building_complex, env_complex]

# 2) Du willst Knoten = jede Cell => also flattenType="Cell"
graph_cells = by_topologies(big_comps, flatten=True, flattenType="Cell")

# 3) ODER du willst Knoten = jede Face => flattenType="Face"
graph_faces = by_topologies(big_comps, flatten=True, flattenType="Face")

Topology.Show(env_complex, graph_cells, graph_faces, renderer=renderer)